# Gazete Tiraj Verileri
Bu script, haftalık gazete satış rakamlarını çekmektedir. Aşağıda verilen kaynakta haftalık olarak en çok satana göre 20 gazete sıralanmaktadır.

**Senaryo**: Sayfada veri ajax ile güncellenmektedir. Sitenin sağ üst köşesinde yer alan tarih seçici ile hafta değiştirilir. Yıl içerisindeki her pazartesi seçilerek sayfada bulunan tablo güncellenir. Güncellemeden sonra sayfadaki tablo çekilerek DataFrame'e dönüştürülür ve bir listeye eklenir. Bir sonraki Pazartesi'ye geçilir.

**Kapsam**: Görünüşe göre web sitesinde 2015'e kadar veri bulunmakta. 2015-2019 aralığındaki veriler yıllık olarak çekilmiştir.

**Başlıklar**

*   date (Tarih)
*   newspaper (Gazete)
*   circulation (Tiraj)

**Erişim tarihi**: 01.11.2019

**Kaynak**: gazetetirajları.com

**Not**: Gazete tirajları resmi bir kaynaktan yapısal bir şekilde paylaşılmadığından, en sağlıklı kaynak olarak bu web sitesi seçilmiştir. Barındırdığı satış rakamlarının güvenilirliği test edilememektedir.

Umut Irmaksever @ AVVG

Gerekli paketler yüklenir ve Selenium instance'ı başlatılır, web sayfasına gidilir.

In [2]:
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!pip install selenium html5lib bs4
!pip install BeautifulSoup4
!pip install html5lib
!pip install matplotlib
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By as By

import html5lib
import pandas as pd

import datetime
from time import sleep

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--window-size=1920,1080")
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
page = wd.get("http://gazetetirajlari.com/HaftalikTirajlar.aspx")

'apt' is not recognized as an internal or external command,
operable program or batch file.
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Bir yıl içerisindeki tüm pazartesileri bir listeye kaydedelim.

In [3]:
def allmondays(year):
    # Yılın ilk günü
    d = datetime.date(year, 1, 1)
    # Pazartesi'yi bul
    d += datetime.timedelta(days=7-d.weekday())
    # Yıl aynı olduğu sürece 7 gün artırarak devam et 
    while d.year == year:
        yield d
        d+= datetime.timedelta(days=7)
    return(d)
# Listedeki tüm tarih objelerini istenen formata sok-
def get_mondays_list(year):
    mondays = allmondays(year)
    mondays = list(map(lambda x: x.strftime("%d.%m.%Y"), mondays))
    return mondays


Sayfadaki veri tablosunu bulup DataFrame'e çevirelim.

In [4]:
def get_circulation_data():
    data_table_div = wd.find_element_by_id("divHaftalikTirajAra")
    week = data_table_div.find_elements_by_tag_name("h3")[0].text
    data_table = data_table_div.find_elements_by_tag_name("table")[0].get_attribute("outerHTML")
    data_table_pd = pd.read_html(data_table, flavor="bs4")
    data_table_pd = data_table_pd[0]
    return data_table_pd

Sayfadaki veri tablosunun üstünde başlık olarak o haftanın tarih aralığı belirtiliyor. Doğrulama amaçlı kullanılabilir. İkinci fonksiyon ise datepicker widget'ına istenen tarihi girip butona basıyor.

In [5]:
def get_current_week():
    week = wd.find_element_by_id("divHaftalikTirajAra").find_elements_by_tag_name("h3")[0].text
    return week
def change_date(date):
    date_textfield = wd.find_element_by_id("txtHaftalikTirajTarih")
    date_textfield.clear()
    # date_textfield.click()
    date_textfield.send_keys(date)
    wd.find_element_by_id("btnHaftalikTirajAra").click()
    # Tarih değiştirildikten sonra bir "Yükleniyor" mesajıyla bir modal çıkıyor.
    # Scriptin sekteye uğramaması için bu modal kapanana kadar bekle.
    element = WebDriverWait(wd, 10).until(
        EC.invisibility_of_element_located((By.ID, 'loading')))

Aşağıdaki helper (yardımcı) fonksiyon ile bir ya da daha fazla yıl içerisindeki Pazartesiler datepicker'da deneniyor ve o hafta için veri olup olmadığı belirleniyor. Veri bulunan haftalar "success_weeks" listesine, olmayan haftalar ise "fail_weeks" listesine kaydediliyor.

In [10]:
weeks_dict = {}
years = range(2019, 2020)
success_weeks = []
fail_weeks = []
def check_if_week_exists(years):
    for year in years:
        mondays_list = get_mondays_list(year)
        for week in mondays_list:
            change_date(week)
            try:
                week_from_website = get_current_week()
                print(week, week_from_website)
                # Hem python tarafından sağlanan tarih, hem de web sitesinden
                # alınan hafta stringini kaydet.
                success_weeks.append([week, week_from_website])
            except IndexError:
                print("nonexistent")
                fail_weeks.append([week, "fail"])
check_if_week_exists(years)

07.01.2019 07 - 13 OCAK 2019
14.01.2019 14 - 20 OCAK 2019
21.01.2019 21 - 27 OCAK 2019
28.01.2019 28 OCAK - 03 ŞUBAT 2019
04.02.2019 04 - 10 ŞUBAT 2019
11.02.2019 11 - 17 ŞUBAT 2019
18.02.2019 18 - 24 ŞUBAT 2019
25.02.2019 25 ŞUBAT - 03 MART 2019
04.03.2019 04 - 10 MART 2019
11.03.2019 11 - 17 MART 2019
18.03.2019 18 - 24 MART 2019
25.03.2019 25 - 31 MART 2019
01.04.2019 01 - 07 NISAN 2019
08.04.2019 08 - 14 NISAN 2019
15.04.2019 15 - 21 NISAN 2019
22.04.2019 22 - 28 NISAN 2019
29.04.2019 29 NISAN - 05 MAYIS 2019
06.05.2019 06 - 12 MAYIS 2019
13.05.2019 13 - 19 MAYIS 2019
20.05.2019 20 - 26 MAYIS 2019
27.05.2019 27 MAYIS - 02 HAZIRAN 2019
03.06.2019 03 - 09 HAZIRAN 2019
10.06.2019 10 - 16 HAZIRAN 2019
17.06.2019 17 - 23 HAZIRAN 2019
24.06.2019 24 - 30 HAZIRAN 2019
01.07.2019 01 - 07 TEMMUZ 2019
08.07.2019 08 - 14 TEMMUZ 2019
15.07.2019 15 - 21 TEMMUZ 2019
22.07.2019 22 - 28 TEMMUZ 2019
29.07.2019 29 TEMMUZ - 04 AĞUSTOS 2019
05.08.2019 05 - 11 AĞUSTOS 2019
12.08.2019 12 - 18 AĞUSTOS 201

Veri olmayan haftaları kaydedelim.

In [11]:
# Fail olan haftaları bir dosyaya yazmak için aşağıdaki dosya ismini değiştirebilirsiniz.
with open("fails_2019_2.txt", mode="w+")as file:
    for fail in fail_weeks:
        file.write(str(fail) + "\n")
    file.close()

Her hafta için veri tablosunu bir dict objesine kaydedelim.

In [12]:
weekly_circulation_list = {}
for week in success_weeks:
    print(week)
    change_date(week[0])
    sleep(0.5)
    circulation_data = get_circulation_data()
    # print(circulation_data)
    weekly_circulation_list[week[0]] = circulation_data

['07.01.2019', '07 - 13 OCAK 2019']
['14.01.2019', '14 - 20 OCAK 2019']
['21.01.2019', '21 - 27 OCAK 2019']
['28.01.2019', '28 OCAK - 03 ŞUBAT 2019']
['04.02.2019', '04 - 10 ŞUBAT 2019']
['11.02.2019', '11 - 17 ŞUBAT 2019']
['18.02.2019', '18 - 24 ŞUBAT 2019']
['25.02.2019', '25 ŞUBAT - 03 MART 2019']
['04.03.2019', '04 - 10 MART 2019']
['11.03.2019', '11 - 17 MART 2019']
['18.03.2019', '18 - 24 MART 2019']
['25.03.2019', '25 - 31 MART 2019']
['01.04.2019', '01 - 07 NISAN 2019']
['08.04.2019', '08 - 14 NISAN 2019']
['15.04.2019', '15 - 21 NISAN 2019']
['22.04.2019', '22 - 28 NISAN 2019']
['29.04.2019', '29 NISAN - 05 MAYIS 2019']
['06.05.2019', '06 - 12 MAYIS 2019']
['13.05.2019', '13 - 19 MAYIS 2019']
['20.05.2019', '20 - 26 MAYIS 2019']
['27.05.2019', '27 MAYIS - 02 HAZIRAN 2019']
['03.06.2019', '03 - 09 HAZIRAN 2019']
['10.06.2019', '10 - 16 HAZIRAN 2019']
['17.06.2019', '17 - 23 HAZIRAN 2019']
['24.06.2019', '24 - 30 HAZIRAN 2019']
['01.07.2019', '01 - 07 TEMMUZ 2019']
['08.07.2019

Bir yıldaki tüm veri olan haftaların bulunduğu dict objesindeki değerler DataFrame şeklinde. Web sayfasındaki tabloyu bire bir kaydettiğimiz için sadece ihtiyacımız olanları alalım ve DataFrame listesini düzleştirelim.

In [13]:
rows_list = []
for circulation_table in weekly_circulation_list.keys():
    week = circulation_table
    table = weekly_circulation_list[circulation_table]
    for row in table.index:
        newspaper_name = table["GAZETE ADI"][row]
        sales = table["GÜNCEL"][row]
        rows_list.append([week, newspaper_name, sales])
        print(week, newspaper_name, sales)
    print(circulation_table)

07.01.2019 SABAH 274.631
07.01.2019 HÜRRİYET 258.178
07.01.2019 SÖZCÜ 251.476
07.01.2019 POSTA 192.729
07.01.2019 TÜRKİYE 132.87
07.01.2019 MİLLİYET 128.627
07.01.2019 YENİ ŞAFAK 110.045
07.01.2019 TAKVİM 105.948
07.01.2019 GÜNEŞ 103.31700000000001
07.01.2019 AKŞAM 103.28
07.01.2019 STAR 100.874
07.01.2019 P.FOTOMAÇ 80.97
07.01.2019 FANATİK 75.429
07.01.2019 KORKUSUZ 53.67
07.01.2019 AYDINLIK 51.361999999999995
07.01.2019 YENİ ASIR 50.942
07.01.2019 YENİ AKİT 50.463
07.01.2019 CUMHURİYET 32.169000000000004
07.01.2019 DOĞRU HABER 14.35
07.01.2019 KARAR 11.039000000000001
07.01.2019
14.01.2019 SABAH 274.209
14.01.2019 HÜRRİYET 262.306
14.01.2019 SÖZCÜ 250.74200000000002
14.01.2019 POSTA 189.082
14.01.2019 TÜRKİYE 133.798
14.01.2019 MİLLİYET 128.787
14.01.2019 YENİ ŞAFAK 109.931
14.01.2019 TAKVİM 106.146
14.01.2019 GÜNEŞ 103.369
14.01.2019 AKŞAM 102.602
14.01.2019 STAR 100.749
14.01.2019 P.FOTOMAÇ 79.706
14.01.2019 FANATİK 77.811
14.01.2019 KORKUSUZ 54.542
14.01.2019 AYDINLIK 51.608999999

Düzleştirilmiş listeyi tekrardan DataFrame'e çevirerek csv formatında kaydedelim.

In [15]:
rows_list_pd = pd.DataFrame(rows_list, columns=["date", "newspaper", "circulation"])
rows_list_pd.to_csv("tr_newspapers_2019.csv")
rows_list_pd

,date,newspaper,circulation
0,07.01.2019,SABAH,274.631
1,07.01.2019,HÜRRİYET,258.178
2,07.01.2019,SÖZCÜ,251.476
3,07.01.2019,POSTA,192.729
4,07.01.2019,TÜRKİYE,132.870
...,...,...,...
1030,16.12.2019,YENİ BİRLİK,32.665
1031,16.12.2019,CUMHURİYET,28.694
1032,16.12.2019,AYDINLIK,14.391
1033,16.12.2019,DOĞRU HABER,13.038
